In [1]:
### Notebook 4: Model Training & Evaluation
### Project: Churn Prevention System
### This notebook trains and evaluates machine learning models
### NOTE: Run AFTER Notebook 3 (Feature Engineering)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (classification_report, confusion_matrix, 
                             roc_auc_score, roc_curve, precision_recall_curve,
                             average_precision_score)
import xgboost as xgb
import joblib
import warnings
warnings.filterwarnings('ignore')

### Random seed for reproducibility
### Why 42? It's a convention (from "Hitchhiker's Guide to the Galaxy")
### Makes results reproducible - same "random" splits every time
np.random.seed(42)

print("=" * 70)
print("CHURN PREDICTION MODEL TRAINING & EVALUATION")
print("=" * 70)

### =============================================================================
### PART 1: LOAD ENGINEERED DATA
### =============================================================================

print("\n" + "=" * 70)
print("PART 1: LOADING ENGINEERED DATA")
print("=" * 70)

### Load the ENGINEERED dataset (from Notebook 3)
df = pd.read_csv('../Datasets/customer_churn_engineered.csv')
print(f"✅ Loaded engineered dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")

### Load feature metadata
import pickle
with open('../Datasets/feature_info.pkl', 'rb') as f:
    feature_info = pickle.load(f)

modeling_features = feature_info['modeling_features']
print(f"✅ Loaded {len(modeling_features)} features for modeling")

### Display feature categories
print(f"\n📊 Feature Breakdown:")
print(f"   • Total features: {len(df.columns)}")
print(f"   • Modeling features: {len(modeling_features)}")
print(f"   • Target variable: churned")

### =============================================================================
### PART 2: PREPARE DATA FOR MODELING
### =============================================================================

print("\n" + "=" * 70)
print("PART 2: PREPARING DATA FOR MACHINE LEARNING")
print("=" * 70)

### Select features and target
X = df[modeling_features].copy()
y = df['churned'].copy()

print(f"\nFeature Matrix (X): {X.shape}")
print(f"Target Vector (y): {y.shape}")
print(f"\nClass Distribution:")
print(f"  • Retained (0): {(y==0).sum():,} ({(y==0).mean()*100:.1f}%)")
print(f"  • Churned (1): {(y==1).sum():,} ({(y==1).mean()*100:.1f}%)")

### Check for any missing values
if X.isnull().sum().sum() > 0:
    print("\n⚠️ Handling missing values...")
    X = X.fillna(X.median())
    print("✅ Missing values filled with median")

### Train-test split with stratification
### Stratification ensures both sets have same churn rate
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,         # 80% train, 20% test
    random_state=42,       # Reproducibility
    stratify=y             # Keep same churn rate in both sets
)

print(f"\n✅ Data Split Complete:")
print(f"   Training set: {X_train.shape[0]:,} samples ({y_train.mean()*100:.1f}% churn)")
print(f"   Test set: {X_test.shape[0]:,} samples ({y_test.mean()*100:.1f}% churn)")

### Scale features for algorithms that need it
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Features scaled using StandardScaler")
print("   (Important for Logistic Regression, not needed for tree-based models)")

### =============================================================================
### PART 3: TRAIN MULTIPLE MODELS
### =============================================================================

print("\n" + "=" * 70)
print("PART 3: TRAINING MULTIPLE ML MODELS")
print("=" * 70)

### Define models with hyperparameters
models = {
    'Logistic Regression': LogisticRegression(
        random_state=42, 
        max_iter=1000,
        class_weight='balanced'  # Handles class imbalance
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=100,        # 100 trees
        random_state=42,
        max_depth=10,            # Prevents overfitting
        min_samples_split=20,    # Min samples to split node
        class_weight='balanced'
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=100,
        random_state=42,
        learning_rate=0.1,       # Step size
        max_depth=5
    ),
    'XGBoost': xgb.XGBClassifier(
        n_estimators=100,
        random_state=42,
        learning_rate=0.1,
        max_depth=5,
        scale_pos_weight=len(y_train[y_train==0])/len(y_train[y_train==1]),  # Handle imbalance
        eval_metric='logloss'
    )
}

results = {}

print("\n🚀 Training models (this takes ~2-3 minutes)...\n")

for name, model in models.items():
    print(f"{'='*70}")
    print(f"Training: {name}")
    print(f"{'='*70}")
    
    ### Train model
    if name == 'Logistic Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    ### Calculate metrics
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    ### Store results
    results[name] = {
        'model': model,
        'auc_score': auc_score,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    
    print(f"\n✅ Training Complete! AUC-ROC: {auc_score:.4f}")
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=['Retained', 'Churned']))

### =============================================================================
### PART 4: MODEL COMPARISON
### =============================================================================

print("\n" + "=" * 70)
print("PART 4: MODEL PERFORMANCE COMPARISON")
print("=" * 70)

### Create comparison dataframe
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'AUC Score': [results[m]['auc_score'] for m in results.keys()]
}).sort_values('AUC Score', ascending=False)

print("\n🏆 Model Performance Ranking:")
print(comparison_df.to_string(index=False))

### Select best model
best_model_name = comparison_df.iloc[0]['Model']
best_model = results[best_model_name]['model']
best_auc = comparison_df.iloc[0]['AUC Score']

print(f"\n🥇 BEST MODEL: {best_model_name}")
print(f"   AUC Score: {best_auc:.4f}")

### =============================================================================
### PART 5: DETAILED EVALUATION OF BEST MODEL
### =============================================================================

print("\n" + "=" * 70)
print("PART 5: DETAILED EVALUATION - BEST MODEL")
print("=" * 70)

best_predictions = results[best_model_name]['predictions']
best_probabilities = results[best_model_name]['probabilities']

### Confusion Matrix
cm = confusion_matrix(y_test, best_predictions)
tn, fp, fn, tp = cm.ravel()

print("\n📊 Confusion Matrix:")
print(f"                 Predicted")
print(f"              Retained  Churned")
print(f"Actual Retained   {cm[0,0]:4d}    {cm[0,1]:4d}")
print(f"       Churned    {cm[1,0]:4d}    {cm[1,1]:4d}")

### Calculate metrics
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
accuracy = (tp + tn) / (tp + tn + fp + fn)

print(f"\n📈 Performance Metrics:")
print(f"  • Accuracy: {accuracy:.3f} ({accuracy*100:.1f}% correct overall)")
print(f"  • Precision: {precision:.3f} (of predicted churns, {precision*100:.1f}% were correct)")
print(f"  • Recall: {recall:.3f} (caught {recall*100:.1f}% of actual churns)")
print(f"  • Specificity: {specificity:.3f} (correctly identified {specificity*100:.1f}% retained)")
print(f"  • F1-Score: {f1:.3f} (harmonic mean of precision & recall)")
print(f"  • AUC-ROC: {best_auc:.3f} (overall ranking ability)")

### Business interpretation
print(f"\n💼 Business Impact:")
print(f"  • Customers flagged as high-risk: {(best_predictions == 1).sum():,}")
print(f"  • Actual churners caught: {tp:,} (out of {tp+fn:,} total)")
print(f"  • False alarms: {fp:,} ({fp/(tp+fp)*100:.1f}% of alerts)")
print(f"  • Missed churners: {fn:,} ({fn/(tp+fn)*100:.1f}% of churners)")

### Cost-benefit analysis
avg_intervention_cost = 50  # $50 per customer intervention
avg_customer_value = 100    # $100 Monthly Reoccuring Revenue

saved_revenue = tp * avg_customer_value
intervention_cost = (tp + fp) * avg_intervention_cost
net_benefit = saved_revenue - intervention_cost

print(f"\n💰 Financial Impact (estimated):")
print(f"  • Saved revenue: ${saved_revenue:,} ({tp} customers × ${avg_customer_value})")
print(f"  • Intervention cost: ${intervention_cost:,} ({tp+fp} interventions × ${avg_intervention_cost})")
print(f"  • Net benefit: ${net_benefit:,}")
print(f"  • ROI: {(net_benefit/intervention_cost)*100:.0f}%")

### =============================================================================
### PART 6: COMPREHENSIVE VISUALIZATIONS (6 CHARTS)
### =============================================================================

print("\n" + "=" * 70)
print("PART 6: CREATING VISUALIZATIONS")
print("=" * 70)

fig = plt.figure(figsize=(20, 12))

### 1. Model Comparison Bar Chart
ax1 = plt.subplot(2, 3, 1)
colors = ['#FF6B6B' if i > 0 else '#4ECDC4' for i in range(len(comparison_df))]
comparison_df.plot(x='Model', y='AUC Score', kind='barh', ax=ax1, legend=False, color=colors)
ax1.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax1.set_xlabel('AUC-ROC Score')
ax1.set_xlim(0, 1)
for i, v in enumerate(comparison_df['AUC Score']):
    ax1.text(v + 0.01, i, f'{v:.4f}', va='center', fontweight='bold')

### 2. ROC Curve for all models
ax2 = plt.subplot(2, 3, 2)
for name, result in results.items():
    fpr, tpr, _ = roc_curve(y_test, result['probabilities'])
    auc = result['auc_score']
    ax2.plot(fpr, tpr, linewidth=2, label=f'{name} (AUC={auc:.3f})')
ax2.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
ax2.set_title('ROC Curves - All Models', fontsize=14, fontweight='bold')
ax2.set_xlabel('False Positive Rate')
ax2.set_ylabel('True Positive Rate (Recall)')
ax2.legend(loc='lower right', fontsize=9)
ax2.grid(alpha=0.3)

### 3. Precision-Recall Curve
ax3 = plt.subplot(2, 3, 3)
precision_curve, recall_curve, thresholds = precision_recall_curve(y_test, best_probabilities)
avg_precision = average_precision_score(y_test, best_probabilities)
ax3.plot(recall_curve, precision_curve, linewidth=2, color='#FF6B6B')
ax3.fill_between(recall_curve, precision_curve, alpha=0.3, color='#FF6B6B')
ax3.set_title(f'Precision-Recall Curve\n(Avg Precision: {avg_precision:.3f})',
             fontsize=14, fontweight='bold')
ax3.set_xlabel('Recall (Sensitivity)')
ax3.set_ylabel('Precision')
ax3.grid(alpha=0.3)

### 4. Confusion Matrix Heatmap
ax4 = plt.subplot(2, 3, 4)
sns.heatmap(cm, annot=True, fmt='d', cmap='RdYlGn_r', ax=ax4,
            xticklabels=['Retained', 'Churned'],
            yticklabels=['Retained', 'Churned'],
            cbar_kws={'label': 'Count'})
ax4.set_title('Confusion Matrix', fontsize=14, fontweight='bold')
ax4.set_ylabel('Actual')
ax4.set_xlabel('Predicted')

### Add percentages as text
for i in range(2):
    for j in range(2):
        text_color = 'white' if cm[i, j] > cm.max() / 2 else 'black'
        pct = cm[i, j] / cm.sum() * 100
        ax4.text(j + 0.5, i + 0.7, f'({pct:.1f}%)', 
                ha='center', va='center', color=text_color, fontsize=10)

### 5. Probability Distribution
ax5 = plt.subplot(2, 3, 5)
retained_probs = best_probabilities[y_test == 0]
churned_probs = best_probabilities[y_test == 1]
ax5.hist(retained_probs, bins=50, alpha=0.6, label='Retained (Actual)', color='#4ECDC4', density=True)
ax5.hist(churned_probs, bins=50, alpha=0.6, label='Churned (Actual)', color='#FF6B6B', density=True)
ax5.axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Decision Threshold')
ax5.set_title('Predicted Churn Probability Distribution', fontsize=14, fontweight='bold')
ax5.set_xlabel('Predicted Churn Probability')
ax5.set_ylabel('Density')
ax5.legend()
ax5.grid(alpha=0.3)

### 6. Threshold Analysis (Precision vs Recall tradeoff)
ax6 = plt.subplot(2, 3, 6)
### Calculate metrics at different thresholds
thresholds_test = np.linspace(0, 1, 100)
precisions = []
recalls = []
f1_scores = []

for thresh in thresholds_test:
    y_pred_thresh = (best_probabilities >= thresh).astype(int)
    cm_thresh = confusion_matrix(y_test, y_pred_thresh)
    if cm_thresh.ravel()[3] + cm_thresh.ravel()[1] > 0:  # Avoid division by zero
        prec = cm_thresh.ravel()[3] / (cm_thresh.ravel()[3] + cm_thresh.ravel()[1])
        rec = cm_thresh.ravel()[3] / (cm_thresh.ravel()[3] + cm_thresh.ravel()[2])
        precisions.append(prec)
        recalls.append(rec)
        if prec + rec > 0:
            f1_scores.append(2 * (prec * rec) / (prec + rec))
        else:
            f1_scores.append(0)
    else:
        precisions.append(0)
        recalls.append(0)
        f1_scores.append(0)

ax6.plot(thresholds_test, precisions, label='Precision', linewidth=2, color='#4ECDC4')
ax6.plot(thresholds_test, recalls, label='Recall', linewidth=2, color='#FF6B6B')
ax6.plot(thresholds_test, f1_scores, label='F1-Score', linewidth=2, color='#FFA07A', linestyle='--')
ax6.axvline(x=0.5, color='black', linestyle=':', linewidth=2, label='Default Threshold (0.5)')
ax6.set_title('Threshold Impact on Metrics', fontsize=14, fontweight='bold')
ax6.set_xlabel('Classification Threshold')
ax6.set_ylabel('Score')
ax6.legend()
ax6.grid(alpha=0.3)
ax6.set_xlim(0, 1)
ax6.set_ylim(0, 1)

plt.tight_layout()
plt.savefig('../Datasets/model_evaluation_comprehensive.png', dpi=300, bbox_inches='tight')
print("\n✅ Saved: ../data/model_evaluation_comprehensive.png (6 visualizations)")
plt.close()

### =============================================================================
### PART 7: FEATURE IMPORTANCE ANALYSIS
### =============================================================================

print("\n" + "=" * 70)
print("PART 7: FEATURE IMPORTANCE ANALYSIS")
print("=" * 70)

if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
    feature_importance = pd.DataFrame({
        'feature': modeling_features,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    print("\n🎯 Top 15 Most Important Features:")
    for i, row in feature_importance.head(15).iterrows():
        print(f"  {i+1:2d}. {row['feature']:40s} {row['importance']:.4f}")
    
    ### Visualize top 20
    plt.figure(figsize=(12, 10))
    top_20 = feature_importance.head(20)
    plt.barh(range(len(top_20)), top_20['importance'], color='#45B7D1')
    plt.yticks(range(len(top_20)), top_20['feature'])
    plt.xlabel('Importance Score', fontsize=12)
    plt.title('Top 20 Features for Churn Prediction', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig('../Datasets/feature_importance_detailed.png', dpi=300, bbox_inches='tight')
    print("\n✅ Saved: ../data/feature_importance_detailed.png")
    plt.close()

### =============================================================================
### PART 8: CROSS-VALIDATION
### =============================================================================

print("\n" + "=" * 70)
print("PART 8: CROSS-VALIDATION FOR ROBUSTNESS CHECK")
print("=" * 70)

print("\nPerforming 5-fold cross-validation on best model...")
print("(This validates model performance is consistent across different data splits)")

cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='roc_auc')

print(f"\n📊 Cross-Validation Results:")
print(f"  • Fold scores: {[f'{score:.4f}' for score in cv_scores]}")
print(f"  • Mean AUC: {cv_scores.mean():.4f}")
print(f"  • Std Dev: {cv_scores.std():.4f}")
print(f"  • Min: {cv_scores.min():.4f} | Max: {cv_scores.max():.4f}")

if cv_scores.std() < 0.05:
    print(f"  ✅ Low variance - model is stable!")
else:
    print(f"  ⚠️ High variance - model performance varies across folds")

### =============================================================================
### PART 9: SAVE MODEL ARTIFACTS
### =============================================================================

print("\n" + "=" * 70)
print("PART 9: SAVING MODEL ARTIFACTS")
print("=" * 70)

### Save best model
model_path = r'../ML Models/churn_model.pkl'
joblib.dump(best_model, model_path)
print(f"✅ Saved model: {model_path}")

### Save scaler
scaler_path = r'../ML Models/scaler.pkl'
joblib.dump(scaler, scaler_path)
print(f"✅ Saved scaler: {scaler_path}")

### Save comprehensive model info
model_info_save = {
    'model_type': best_model_name,
    'auc_score': best_auc,
    'feature_columns': modeling_features,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'accuracy': accuracy,
    'cv_scores': cv_scores.tolist(),
    'confusion_matrix': cm.tolist(),
    'threshold': 0.5
}

info_path = r'../ML Models/model_info.pkl'
joblib.dump(model_info_save, info_path)
print(f"✅ Saved model info: {info_path}")

### Save test predictions
predictions_df = pd.DataFrame({
    'customer_id': df.iloc[X_test.index]['customer_id'].values,
    'actual_churn': y_test.values,
    'predicted_churn': best_predictions,
    'churn_probability': best_probabilities
})
predictions_path = '../Datasets/test_predictions.csv'
predictions_df.to_csv(predictions_path, index=False)
print(f"✅ Saved predictions: {predictions_path}")

### =============================================================================
### SUMMARY
### =============================================================================

print("\n" + "=" * 70)
print("✅ MODEL TRAINING COMPLETE!")
print("=" * 70)

print(f"""
Best Model Performance:
  🥇 Model: {best_model_name}
  📊 AUC-ROC: {best_auc:.4f}
  🎯 Precision: {precision:.3f} ({precision*100:.1f}% of alerts are correct)
  🎯 Recall: {recall:.3f} (catches {recall*100:.1f}% of churners)
  🎯 F1-Score: {f1:.3f}
  🎯 Accuracy: {accuracy:.3f}
  
Business Impact:
  • Can identify {recall*100:.0f}% of churners in advance
  • {fp:,} false positives out of {tp+fp:,} total alerts
  • Estimated net benefit: ${net_benefit:,}
  • ROI: {(net_benefit/intervention_cost)*100:.0f}%
  
Cross-Validation:
  • Mean AUC: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}
  • Model is {'stable ✅' if cv_scores.std() < 0.05 else 'variable ⚠️'}

Files Created:
  1. churn_model.pkl (trained model)
  2. scaler.pkl (feature scaler)
  3. model_info.pkl (metadata)
  4. test_predictions.csv (test set results)
  5. model_evaluation_comprehensive.png (6 charts)
  6. feature_importance_detailed.png (top 20 features)
""")

print("\n🚀 Ready for Model Interpretation!")
print("=" * 70)

CHURN PREDICTION MODEL TRAINING & EVALUATION

PART 1: LOADING ENGINEERED DATA
✅ Loaded engineered dataset: 5,000 rows × 54 columns
✅ Loaded 44 features for modeling

📊 Feature Breakdown:
   • Total features: 54
   • Modeling features: 44
   • Target variable: churned

PART 2: PREPARING DATA FOR MACHINE LEARNING

Feature Matrix (X): (5000, 44)
Target Vector (y): (5000,)

Class Distribution:
  • Retained (0): 3,820 (76.4%)
  • Churned (1): 1,180 (23.6%)

✅ Data Split Complete:
   Training set: 4,000 samples (23.6% churn)
   Test set: 1,000 samples (23.6% churn)
✅ Features scaled using StandardScaler
   (Important for Logistic Regression, not needed for tree-based models)

PART 3: TRAINING MULTIPLE ML MODELS

🚀 Training models (this takes ~2-3 minutes)...

Training: Logistic Regression

✅ Training Complete! AUC-ROC: 1.0000

Classification Report:
              precision    recall  f1-score   support

    Retained       1.00      1.00      1.00       764
     Churned       1.00      1.00  